# **Support Vector Machine(SVM)- nonLineal algorithms**

Scikit-learn ofrece una implementación basada en **LIBSVM** (biblioteca de implementaciones de clasificación y regresión SVM), y *LIBLINEAR* (biblioteca escalable para la clasificación lineal ideal de grandes conjuntos de datos, especialmente cualquier texto disperso). Ambas han sido escritas en C++ con una API en C para interactuar con otros lenguajes.

La API en C explica bien dos necesidades complicadas para que funcionen de forma óptima bajo el
scikit-learn de Python: 
*   LIBSVM, al funcionar, necesita reservar algo de memoria para las operaciones del **kernel**. El parámetro *cache_size* es el encargado de eso , por defecto es 200 megabytes, es aconsejable subirlo a 500 o 1000, dependiendo de los recursos disponibles
*   Ambos esperan un ndarray de NumPy o un SciPy sparse.csr_matrix (un tipo de matriz dispersa optimizada para filas),
preferiblemente con tipo *float64*.
---
Ni *LIBSVM* ni *LIBLINEAR* ofrecen una implementación capaz de manejar grandes
conjuntos de datos. **SGDClassifier** y **SGDRegressor** son las clases de scikit-learn que
pueden producir una solución en un tiempo computacional razonable

## SVM for classification
Posee las siguientes implementaciones: 
* `sklearn.svm.SVC`: implementacion basada en  LIBSVM  para la clasificación lineal y kernel funciona bien hasta 10 000 datos ya que tiene complejidad cubica. 
* `sklearn.svm.NuSVC`: similar que la versión .SVC
* `sklearn.svm.OneClassSVM`: Detección no supervisada de valores atípicos
* `sklearn.svm.LinearSVC`: Basado en LIBLINEAR es un clasificador lineal binario y multiclase

Los datos q usaremos son una serie temporal de 50.000 muestras producidas por un sistema físico de un motor de combustión interna de 10 cilindros.Nuestro objetivo es binario: encendido normal del motor o fallo deencendido. 

Utilizaremos el conjunto de datos tal y como se ha recuperado del sitio web de *LIBSVM* utilizando el scripts *4_0download_data.ipynb*. El archivo de datos está en el formato LIBSVM y está comprimido por Bzip2

In [26]:
from sklearn.datasets import load_svmlight_file
X_train, y_train = load_svmlight_file('resources/ijcnn1.bz2')
first_rows = 2500
X_train, y_train = X_train[:first_rows,:], y_train[:first_rows]
print(X_train[0],'\n',type(X_train))
X_train.shape

  (0, 5)	1.0
  (0, 10)	-0.731854
  (0, 11)	0.173431
  (0, 12)	0.0
  (0, 13)	0.00027
  (0, 14)	0.011684
  (0, 15)	-0.011052
  (0, 16)	0.024452
  (0, 17)	0.008337
  (0, 18)	0.001324
  (0, 19)	0.025544
  (0, 20)	-0.040728
  (0, 21)	-0.00081 
 <class 'scipy.sparse._csr.csr_matrix'>


(2500, 22)

In [27]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
hypothesis = SVC(kernel='rbf', random_state=101)

scores = cross_val_score(hypothesis, X_train, y_train, cv=5, scoring='accuracy')

print ("SVC with rbf kernel -> cross validation accuracy: \
mean = %0.3f std = %0.3f" % (np.mean(scores), np.std(scores)))

SVC with rbf kernel -> cross validation accuracy: mean = 0.910 std = 0.001


Con una muestra de tamaño 15 000 obtuvimos en un timepo mucho mayor (debido a la complejidad cubuca de esta implementacion):  
>SVC with rbf kernel -> cross validation accuracy: mean = 0.944 std = 0.018

Otro ejempo para probar la escalabilidad de la *SVM* en un problema multiclase y a un gran número de casos. El
conjunto de datos Covertype, que vamos a utilizar, presenta como ejemplos un gran número de manchas de bosque de 30x30 metros en Estados  Unidos. Los datos correspondientes se recogen para la tarea de predecir la
especie arbórea dominante de cada parche (tipo de cobertura). Se trata de
un problema de clasificación multiclase (siete tipos de cubierta a predecir).
Cada muestra tiene 54 características, y hay más de 580.000 ejemplos.

Las clases están desequilibradas, ya que hay dos tipos de árboles con la mayoría de los ejemplos.

In [40]:
import numpy as np
import pickle
covertype_dataset = pickle.load(open("resources/covertype_dataset.pickle", "rb"))

covertypes = ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine',
              'Cottonwood/Willow', 'Aspen', 'Douglas-fir', 'Krummholz']

covertype_X = covertype_dataset.data[:25000, :]

# restamosm uno para iniciar las class desde cero
covertype_y = covertype_dataset.target[:25000] - 1


print('sub-sample:', covertype_X.shape)
print('target freq:', list(zip(covertypes, np.bincount(covertype_y)/covertype_X.shape[0])))


sub-sample: (25000, 54)
target freq: [('Spruce/Fir', 0.36428), ('Lodgepole Pine', 0.48488), ('Ponderosa Pine', 0.06332), ('Cottonwood/Willow', 0.0048), ('Aspen', 0.01648), ('Douglas-fir', 0.03116), ('Krummholz', 0.03508)]


Dado que tenemos siete clases, tendremos que entrenar siete clasificadores diferentes centrados en la
predicción de una sola clase frente a las demás (uno frente a otro es el comportamiento por defecto de **LinearSVC** en problemas multiclase). Esto es  todo un reto para muchos algoritmos, teniendo en cuenta que hay 54
variables, pero **LinearSVC** puede demostrar cómo manejarlo en un tiempo razonable:

In [46]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import LinearSVC

hypothesis = LinearSVC(dual=False, class_weight='balanced')

# cross-validation-generator equilibrar los pliegues (es el mismo q usa por defecto cross_val_score)
cv_strata = StratifiedKFold(n_splits=3, shuffle=True, random_state=101)

scores = cross_val_score(hypothesis, covertype_X, covertype_y,
                         cv=cv_strata, scoring='accuracy')
print("LinearSVC -> cross validation accuracy: \
mean = %0.3f std = %0.3f" % (np.mean(scores), np.std(scores)))


LinearSVC -> cross validation accuracy: mean = 0.648 std = 0.024


La precisión resultante es 0.65 , que no es un mal resultado. Sin embargo, deja margen de mejora. Por otra parte, el problema parece  ser no lineal, aunque aplicar el SVM con un kernel no lineal daría lugar a un
proceso de entrenamiento muy largo, ya que el número de observaciones
es grande. Volveremos a plantear este problema en los siguientes ejemplos
utilizando otros algoritmos no lineales para comprobar si podemos mejorar
la puntuación obtenida por **LinearSVC**.

## SVM for regression
Posee las siguientes implementaciones: 
* `sklearn.svm.SVR`: implementacion basada en LIBSVM para la regresión
* `sklearn.svm.NuSVR`: Lo mismo que para .SVR 

Para ofrecer un ejemplo de regresión, nos decidimos por un conjunto de datos sobre los precios inmobiliarios de las viviendas en California 


In [77]:
from sklearn.preprocessing import scale
import pickle
X_train, y_train = pickle.load(open("resources/cadata.pickle", "rb"))
first_rows = 2000

# escalar para evitar la influencia de la diferente escala de las variables originales
X_train = scale(X_train[:first_rows, :].toarray())

# divide por1,000 para hacerla más legible en valores de mil dólares
y_train = y_train[:first_rows]/10**4.0


In [80]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
hypothesis = SVR()

# el signo negativo es sólo un truco computacional utilizado por  scikit- learn
scores = cross_val_score(hypothesis, X_train, y_train, cv=3, scoring='neg_mean_absolute_error')

print("SVR -> cross validation accuracy: mean = %0.3f \
std = %0.3f" % (np.mean(scores), np.std(scores)))

SVR -> cross validation accuracy: mean = -4.652 std = 0.300


## Tuning hyperparameters SVM
Observaciones 
1. Al igual que otros algoritmos de aprendizaje basados en combinaciones lineales, tener variables a diferentes
escalas hace que el algoritmo se vea dominado por las características con
mayor rango o varianza. Es aconsejable escalar todos los datos
 
2.  El algoritmo tiende a favorecer las clases frecuentes. Una solución, aparte del
remuestreo o del downsampling (reducir la clase mayoritaria al mismo
número de la menos frecuente), es ponderar el *parámetro de penalización
C* en función de la frecuencia de la clase (los valores bajos penalizarán más
la clase, los valores altos menos). Hay dos formas de conseguirlo con
respecto a las diferentes implementaciones; primero, está el parámetro
*class_weight* en **SVC** (que puede establecerse con la palabra clave balanced, o
proporcionarse con un diccionario que contenga valores específicos para
cada clase). Luego, también está el parámetro *sample_weight* en el método
.fit() de *SVC*,*NuSVC* ,*SVR* ,*NuSVR* , y **OneClassSVM** (requiere una matriz
unidimensional como entrada, donde cada posición se refiere al peso decada ejemplo de entrenamiento).
---
* *C*: El valor de la penalización. Disminuirlo hace que el margen sea mayor, ignorando así más ruido pero también haciendo que el modelo sea más generalizable. Un valor óptimo puede considerarse normalmente en el rango de `np.logspace(-3,3, 7)`
 
* *kernel*: se puede establecer en lineal, poly, rbf, sigmoid, o un núcleo personalizado (¡para expertos!). El más
utilizado es sin duda *rbf*(radial basis function)
 
* *degree*: Esto funciona con kernel='poly', señalando la dimensionalidad de la expansión polinómica. En cambio, es ignorado por otros núcleos
 
* *gamma*: Un coeficiente para'rbf' , 'poly', y 'sigmoid'. Los valores altos tienden a ajustarse mejor a los datos, pero pueden llevar a un cierto sobreajuste. Intuitivamente, podemos imaginarnos la gamma como la influencia que un solo ejemplo ejerce sobre el modelo. Los valores bajos hacen que la influencia de cada ejemplo se sienta bastante lejos. Como
hay que considerar muchos puntos, la curva de la SVM tenderá a tomar una forma menos influenciada por los puntos locales y el resultado será una curva de contorno mórbida. Los valores altos de gamma, en cambio, hacen que la curva tenga más en cuenta la disposición de los puntos a nivel local. Los resultados suelen estar representados por muchas burbujas pequeñas que explican la influencia ejercida por los puntos locales. El rango de búsqueda sugerido para este hiperparámetro es `np.logspace(-3, 3, 7)`
 
* *nu* : Para la regresión y la clasificación con **NuSVR** y *NuSVC*, este parámetro aproxima los puntos de
entrenamiento que no se clasifican con confianza, es decir, los puntos mal clasificados y los puntos correctos dentro o en el margen. Debe estar en el rango de [0,1], ya que es una proporción relativa a su conjunto de entrenamiento. Al final, actúa como C, con proporciones altas que amplían el margen
 
* *epsilon*: Este parámetro especifica cuánto error va a aceptar el SVR  definiendo un rango grande epsilon donde no se asocia ninguna penalización con respecto al valor verdadero del punto. El rango de búsqueda sugerido es
`np.insert(np.logspace(-4, 2, 7),0,[0])`. 
 
* *penalty*, *loss*, and *dual*: For LinearSVC, these parameters accept the ('l1','squared_hinge',False), ('l2','hinge',True),
('l2','squared_hinge',True), and ('l2','squared_hinge',False)combinations.
 The ('l2','hinge',True) combination is analogous to the SVC (kernel='linear'). 

* El problema *dual* es más fácil de resolver cuando el numero de instancias de entrenamiento es menor que el número de caracteristicas  y ademas permite realizar el **Trick kernel** mientras que el primario no 

In [83]:
# Intentaremos mejorar la precisión inicial de 0,91 en el conjunto de datos IJCNN'01 buscando
# mejores valores de grado,C , y gamma.

from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

X_train, y_train = load_svmlight_file('resources/ijcnn1.bz2')
first_rows = 5000
X_train, y_train = X_train[:first_rows, :], y_train[:first_rows]

hypothesis = SVC(kernel='rbf', random_state=101)

search_dict = {'C': [0.01, 0.1, 1, 10, 100],
               'gamma': [0.1, 0.01, 0.001, 0.0001]}

search_func = RandomizedSearchCV(estimator=hypothesis,
                                 param_distributions=search_dict,
                                 n_iter=10, scoring='accuracy',
                                 n_jobs=-1, refit=True,
                                 cv=5, random_state=101)
search_func.fit(X_train, y_train)
print('Best parameters %s' % search_func.best_params_)
print('Cross validation accuracy: mean = %0.3f' %
      search_func.best_score_)


Best parameters {'gamma': 0.1, 'C': 100}
Cross validation accuracy: mean = 0.997
